# Propositionalization: Traffic near Dodgers' stadium

In this notebbok, we compare getML's FastProp against well-known feature engineering libraries featuretools and tsfresh.

Summary:

- Prediction type: __Regression model__
- Domain: __Transportation__
- Prediction target: __traffic volume__ 
- Source data: __Univariate time series__
- Population size: __47497__

_Author: Dr. Patrick Urbanke_

# Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterward. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

In this notebook, we use traffic data that was collected for the Glendale on ramp for the 101 North freeway in Los Angeles. For further details about the data set refer to [the full notebook](../dodgers.ipynb).

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

# Analysis

Let's get started with the analysis and set-up your session:

In [1]:
import datetime
import gc
import os
from urllib import request
import time

import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import scipy

from utils import FTTimeSeriesBuilder, TSFreshBuilder

from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

For various technical reasons, we want to keep our MyBinder notebook short. That is why we pre-store the features for prophet and tsfresh. However, you are very welcome to try this at home and fully reproduce our results. You can just set the two constants to "True".

In [2]:
import getml

getml.engine.set_project('dodgers')


Connected to project 'dodgers'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the UC Irvine Machine Learning repository:

In [3]:
fname = "Dodgers.data"

if not os.path.exists(fname):
    fname, res = request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/" + fname, 
        fname
    )
    
data_full_pandas = pd.read_csv(fname, header=None)
data_full_pandas.columns = ["ds", "y"]

In [4]:
data_full_pandas["ds"] = [
    datetime.datetime.strptime(dt, "%m/%d/%Y %H:%M") for dt in data_full_pandas["ds"]
]

In [5]:
data_full_pandas

,ds,y
0,2005-04-10 00:00:00,-1
1,2005-04-10 00:05:00,-1
2,2005-04-10 00:10:00,-1
3,2005-04-10 00:15:00,-1
4,2005-04-10 00:20:00,-1
...,...,...
50395,2005-10-01 23:35:00,-1
50396,2005-10-01 23:40:00,-1
50397,2005-10-01 23:45:00,-1
50398,2005-10-01 23:50:00,-1


We also want to separate our data set into a training and testing set:

In [6]:
separation = datetime.datetime.strptime("8/20/2005 0:00", "%m/%d/%Y %H:%M")

data_train_pandas = data_full_pandas[data_full_pandas["ds"] < separation]
data_test_pandas = data_full_pandas[data_full_pandas["ds"] > separation]

### 1.2 Prepare data for getML

In [7]:
data_full = getml.data.DataFrame.from_pandas(data_full_pandas, "data_full")
data_train = getml.data.DataFrame.from_pandas(data_train_pandas, "data_train")
data_test = getml.data.DataFrame.from_pandas(data_test_pandas, "data_test")

In [8]:
for df in [data_full, data_train, data_test]:
    df.set_role("y", getml.data.roles.target)
    df.set_role("ds", getml.data.roles.time_stamp)

In [9]:
data_full

Name,ds,y
Role,time_stamp,target
Units,"time stamp, comparison only",
0,2005-04-10,-1
1,2005-04-10 00:05:00,-1
2,2005-04-10 00:10:00,-1
3,2005-04-10 00:15:00,-1
4,2005-04-10 00:20:00,-1
,...,...
50395,2005-10-01 23:35:00,-1
50396,2005-10-01 23:40:00,-1


### 1.3 Define relational model

To start with relational learning, we need to specify the data model. We manually replicate the appropriate time series structure by setting time series related join conditions (`horizon`, `memory` and `allow_lagged_targets`). This is done abstractly using [Placeholders](https://docs.getml.com/latest/user_guide/data_model/data_model.html#placeholders)

The data model consists of two tables:
* __Population table__ `traffic_{test/train}`: holds target and the contemporarily available time-based components
* __Peripheral table__ `traffic`: same table as the population table
* Join between both placeholders specifies (`horizon`) to prevent leaks and (`memory`) that keeps the computations feasible

In [10]:
population = getml.data.Placeholder('population')

peripheral = getml.data.Placeholder('peripheral')

# 1. The horizon is 1 hour (we predict the traffic volume in one hour). 
# 2. The memory is 2 hours, so we allow the algorithm to 
#    use information from up to 2 hours ago. 
# 3. We allow lagged targets. Thus, the algorithm can 
#    identify autoregressive processes.

population.join(
    peripheral,
    time_stamp='ds',
    horizon=getml.data.time.hours(1),
    memory=getml.data.time.hours(2),
    allow_lagged_targets=True
)

population

placeholder   other placeholder   allow lagged targets   horizon   join keys used   memory   other join keys used   other time stamps used   relationship   time stamps used   upper time stamps used
  0   population    peripheral          True                   3600.0                     7200.0                          ds                       many-to-many   ds

## 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

### 2.1 Propositionalization with getML's FastProp

In [11]:
seasonal = getml.preprocessors.Seasonal()

fast_prop = getml.feature_learning.FastPropModel(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
)

__Build the pipeline__

In [12]:
pipe_fp_fl = getml.pipeline.Pipeline(
    preprocessors=[seasonal],
    feature_learners=[fast_prop],
    population=population,
    tags=["feature learning", "fastprop"],
)

pipe_fp_fl

Pipeline(preprocessors=['Seasonal'],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=[],
         tags=['feature learning', 'fastprop'],
         share_selected_features=0.5)

In [13]:
pipe_fp_fl.check(data_train, [data_full])

Checking data model...
OK.


In [14]:
begin = time.time()

pipe_fp_fl.fit(data_train, [data_full])

fastprop_train = pipe_fp_fl.transform(
    data_train, [data_full], df_name="fastprop_train")

end = time.time()

fastprop_runtime = datetime.timedelta(seconds=end - begin)

Checking data model...
OK.

Preprocessing...
[========================================] 100%

FastProp: Trying 486 features...
[========================================] 100%

Trained pipeline.
Time taken: 0h:2m:9.273252


Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%



In [15]:
fastprop_test = pipe_fp_fl.transform(
    data_test, [data_test], df_name="fastprop_test")


Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%



In [16]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [17]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:13.599406



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['prediction', 'fastprop'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/dodgers/MWQzSR/0/

In [18]:
pipe_fp_pr.score(fastprop_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-17 17:00:18,fastprop_train,y,5.6182,7.7183,0.684
1,2021-05-17 17:00:19,fastprop_test,y,5.8154,7.994,0.6605


### 2.2 Propositionalization with featuretools

In [19]:
dfs_pandas = {}

for df in getml.project.data_frames:
    dfs_pandas[df.name] = df.to_pandas()
    dfs_pandas[df.name]["id"] = 1

In [20]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(hours=1),
    memory=pd.Timedelta(hours=2),
    column_id="id",
    time_stamp="ds",
    target="y",
    allow_lagged_targets=True,
)

In [21]:
featuretools_train = ft_builder.fit(dfs_pandas["data_train"])
featuretools_test = ft_builder.transform(dfs_pandas["data_test"])

featuretools: Trying features...


/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Selecting the best out of 77 features...
Time taken: 0h:9m:3.091081



/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [22]:
df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=data_train.roles
)

df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=data_train.roles
)

In [23]:
df_featuretools_train.set_role(
    df_featuretools_train.unused_names, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.unused_names, getml.data.roles.numerical
)

In [24]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

In [25]:
pipe_ft_pr.check(df_featuretools_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


Checking data model...


In [26]:
pipe_ft_pr.fit(df_featuretools_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


Checking data model...

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:5.665271



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/dodgers/QDe8qQ/0/

In [27]:
pipe_ft_pr.score(df_featuretools_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-17 17:12:21,featuretools_train,y,5.406,7.5485,0.6978
1,2021-05-17 17:12:21,featuretools_test,y,5.8312,8.165,0.6621


### 2.3 Propositionalization with tsfresh

In [28]:
tsfresh_builder = TSFreshBuilder(
    num_features=200, 
    horizon=20,
    memory=60, 
    column_id="id", 
    time_stamp="ds", 
    target="y",
    allow_lagged_targets=True
)

In [29]:
tsfresh_train = tsfresh_builder.fit(dfs_pandas["data_train"])

tsfresh_test = tsfresh_builder.transform(dfs_pandas["data_test"])

Feature Extraction: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]


Selecting the best out of 12 features...
Time taken: 0h:2m:7.847436



Feature Extraction: 100%|██████████| 20/20 [00:10<00:00,  1.97it/s]


In [30]:
df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, 
    name="tsfresh_train", 
    roles=data_train.roles
)

df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, 
    name="tsfresh_test", 
    roles=data_train.roles
)

In [31]:
df_tsfresh_train.set_role(df_tsfresh_train.unused_names, getml.data.roles.numerical)

df_tsfresh_test.set_role(df_tsfresh_test.unused_names, getml.data.roles.numerical)

In [32]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

In [33]:
pipe_tsf_pr.fit(df_tsfresh_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


Checking data model...

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:2.107887



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/dodgers/8Sjnvj/0/

In [34]:
pipe_tsf_pr.score(df_tsfresh_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-17 17:15:15,tsfresh_train,y,6.3146,8.2348,0.6418
1,2021-05-17 17:15:15,tsfresh_test,y,6.7881,8.9132,0.5779


In [35]:
num_features = dict(
    fastprop=486,
    featuretools=77,
    tsfresh=11,
)

runtime_per_feature = [
    fastprop_runtime / num_features['fastprop'],
    ft_builder.runtime / num_features['featuretools'],
    tsfresh_builder.runtime / num_features['tsfresh'],
]

features_per_second = [1.0/r.total_seconds() for r in runtime_per_feature]

speedup_per_feature = [r/runtime_per_feature[0] for r in runtime_per_feature]

comparison = pd.DataFrame(
    dict(
        runtime=[fastprop_runtime, ft_builder.runtime, tsfresh_builder.runtime],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        speedup=[1, ft_builder.runtime/fastprop_runtime, tsfresh_builder.runtime/fastprop_runtime],
        speedup_per_feature=speedup_per_feature,
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared, pipe_tsf_pr.rsquared],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse, pipe_tsf_pr.rmse],
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae, pipe_tsf_pr.mae],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [36]:
comparison

,runtime,num_features,features_per_second,speedup,speedup_per_feature,rsquared,rmse,mae
getML: FastProp,0 days 00:03:00.892329,486,2.686684,1.000000,1.000000,0.660474,7.994006,5.815399
featuretools,0 days 00:09:03.091081,77,0.141781,3.002289,18.949536,0.662080,8.165049,5.831166
tsfresh,0 days 00:02:07.847436,11,0.086040,0.706760,31.225972,0.577854,8.913240,6.788137


In [37]:
# export for further use
comparison.to_csv("comparisons/dodgers.csv")

## 3. Conclusion

You are encouraged to reproduce these results. You will need getML (https://getml.com/product) to do so. You can download it for free.

# Next Steps

This tutorial went showcased another time series application of getML and benchmarked getML against popular time series libraries.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.